In [3]:
import fiona
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

import geopandas as gpd

import numpy as np

import os



def CreateFolder(directory):
    try :
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory.' + directory)
        
        
        
name1   = 'ME'
name2   = 'IR'

path1      = './'+str(name1)
path2     = './'+str(name2)

path_list = sorted([f for f in os.listdir(str(path1)) if f.endswith('.tif')])
CreateFolder(str(path2))

# Read Shape file
with fiona.open("./Iran.shp", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

for i,file in enumerate(path_list):
    # read imagery file
    with rasterio.open(str(path1)+'/'+str(file)) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta
        
    # Save clipped imagery
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(str(path2)+'/IR_'+str(file), "w", **out_meta) as dest:
        dest.write(out_image)